In [1]:
import numpy as np
import math
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

In [19]:
df1 = pd.read_csv('../data/intermediate/building_1166/train_and_building_meta_1166.csv', parse_dates=['timestamp'], 
                  index_col=['timestamp'])
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8784 entries, 2016-01-01 00:00:00 to 2016-12-31 23:00:00
Data columns (total 8 columns):
building_id      8784 non-null int64
meter            8784 non-null int64
meter_reading    8784 non-null float64
site_id          8784 non-null int64
primary_use      8784 non-null object
square_feet      8784 non-null int64
year_built       0 non-null float64
floor_count      0 non-null float64
dtypes: float64(3), int64(4), object(1)
memory usage: 617.6+ KB


In [25]:
# Check all building ids are 1166 then drop column 'building_id'
print('building_id values : {}'.format(df1.building_id.unique()))
df1.drop('building_id', axis=1, inplace=True)
df1.head()

building_id values : [1166]


,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
timestamp,,,,,,,
2016-01-01 00:00:00,0,50.102,13,Office,63771,NaN,NaN
2016-01-01 01:00:00,0,50.102,13,Office,63771,NaN,NaN
2016-01-01 02:00:00,0,50.114,13,Office,63771,NaN,NaN
2016-01-01 03:00:00,0,49.894,13,Office,63771,NaN,NaN
2016-01-01 04:00:00,0,49.601,13,Office,63771,NaN,NaN


In [26]:
# TODO is 63771 big (among buidings and among Offices buidings) ?

In [30]:
# mean square_feet for Offices : 87066 (cf explore_building meta)

In [28]:
# We've seen that we have a an 'Office' building, on site 13, 63771 square feets, no year_built, no floor count.
# Lets drop all that.
df1.drop(['site_id', 'primary_use', 'square_feet', 'year_built', 'floor_count'], axis=1, inplace=True)

In [29]:
df1.head()

,meter,meter_reading
timestamp,,
2016-01-01 00:00:00,0,50.102
2016-01-01 01:00:00,0,50.102
2016-01-01 02:00:00,0,50.114
2016-01-01 03:00:00,0,49.894
2016-01-01 04:00:00,0,49.601


In [41]:
df2 = pd.read_csv('../data/intermediate/building_1166/weather_site_13.csv', parse_dates=['timestamp'], 
                  index_col=['timestamp'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8783 entries, 2016-01-01 00:00:00 to 2016-12-31 23:00:00
Data columns (total 7 columns):
air_temperature       8782 non-null float64
cloud_coverage        4485 non-null float64
dew_temperature       8782 non-null float64
precip_depth_1_hr     8771 non-null float64
sea_level_pressure    8663 non-null float64
wind_direction        8566 non-null float64
wind_speed            8782 non-null float64
dtypes: float64(7)
memory usage: 548.9 KB


In [34]:
df2.head()

,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
timestamp,,,,,,,
2016-01-01 00:00:00,-8.3,8.0,-12.2,NaN,1023.3,250.0,4.1
2016-01-01 01:00:00,-8.3,NaN,-11.7,-1.0,1023.3,240.0,4.1
2016-01-01 02:00:00,-7.8,NaN,-11.1,-1.0,1022.9,240.0,3.1
2016-01-01 03:00:00,-7.8,NaN,-11.1,-1.0,1022.7,250.0,4.1
2016-01-01 04:00:00,-7.2,NaN,-10.6,0.0,1022.8,270.0,4.1


In [35]:
df1.shape

(8784, 2)

In [36]:
df2.shape

(8783, 7)

In [38]:
# Join by timestamp
df = df1.join(df2, on='timestamp', how='left')
#print('non-found site_ids : {}'.format(dfs['site_id'].isna().sum()))
#dfs.head()

In [39]:
df.shape

(8784, 9)

In [40]:
df.head()

,meter,meter_reading,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
timestamp,,,,,,,,,
2016-01-01 00:00:00,0,50.102,-8.3,8.0,-12.2,NaN,1023.3,250.0,4.1
2016-01-01 01:00:00,0,50.102,-8.3,NaN,-11.7,-1.0,1023.3,240.0,4.1
2016-01-01 02:00:00,0,50.114,-7.8,NaN,-11.1,-1.0,1022.9,240.0,3.1
2016-01-01 03:00:00,0,49.894,-7.8,NaN,-11.1,-1.0,1022.7,250.0,4.1
2016-01-01 04:00:00,0,49.601,-7.2,NaN,-10.6,0.0,1022.8,270.0,4.1


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8784 entries, 2016-01-01 00:00:00 to 2016-12-31 23:00:00
Data columns (total 9 columns):
meter                 8784 non-null int64
meter_reading         8784 non-null float64
air_temperature       8782 non-null float64
cloud_coverage        4485 non-null float64
dew_temperature       8782 non-null float64
precip_depth_1_hr     8771 non-null float64
sea_level_pressure    8663 non-null float64
wind_direction        8566 non-null float64
wind_speed            8782 non-null float64
dtypes: float64(8), int64(1)
memory usage: 1006.2 KB


In [43]:
# Which meters ?
df['meter'].unique()

array([0])